In [29]:
import chipwhisperer as cw
scope = cw.scope()
scope.default_setup()
if scope._is_husky:
    scope.adc.samples = 80
else:
    scope.adc.samples = 129
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "disabled"

scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 603673                    to 21315646                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 0                         to 29538459                 
scope.clock.adc_rate                     changed from 0.0                       to 29538459.0               
scope.clock.freq_ct

In [30]:
#TARGET_PLATFORM = 'CW305_100t'
TARGET_PLATFORM = 'CW305_35t'
# TARGET_PLATFORM = 'CW312T_A35'
#TARGET_PLATFORM = 'CW312T_ICE40'

In [ ]:
if TARGET_PLATFORM in ['CW312T_A35', 'CW312T_ICE40']:
    scope.io.hs2 = 'clkgen'
    fpga_id = None # not needed
    if TARGET_PLATFORM == 'CW312T_A35':
        platform = 'ss2_a35'
        scope.gain.db = 45 # this is a good setting for the inductive shunt; if using another, adjust as needed
    else:
        platform = 'ss2_ice40'
        scope.gain.db = 15
else:
    scope.gain.db = 25
    scope.io.hs2 = "disabled"
    platform = 'cw305'
    if TARGET_PLATFORM == 'CW305_100t':
        fpga_id = '100t'
    elif TARGET_PLATFORM == 'CW305_35t':
        fpga_id = '35t'

# On the CW305, setting force=False only programs the FPGA if it is currently unprogrammed, whereas force=True programs the FPGA regardless.
# This option isn't available on the CW312T_A35 or CW312T_ICE40.
target = cw.target(scope, cw.targets.CW305, force=True, fpga_id=fpga_id, platform=platform)



# target = cw.target(None, cw.targets.CW305, force=True, bsfile="/home/boochoo/hqc/dummy-insertion-sparse-polymult/CW305/jupyter/cw305_top.bit")
target = cw.target(None, cw.targets.CW305, force=True, bsfile=open("/home/boochoo/hqc/dummy-insertion-sparse-polymult/CW305/jupyter/cw305_top.bit", "rb"))


USBErrorIO: LIBUSB_ERROR_IO [-1]

In [35]:
target.fpga.FPGAProgram(open("/home/boochoo/hqc/dummy-insertion-sparse-polymult/CW305/jupyter/cw305_top.bit", "rb"))

True

In [4]:
if TARGET_PLATFORM in ['CW305_100t', 'CW305_35t']:
    target.vccint_set(1.0)
    # we only need PLL1:
    target.pll.pll_enable_set(True)
    target.pll.pll_outenable_set(False, 0)
    target.pll.pll_outenable_set(True, 1)
    target.pll.pll_outenable_set(False, 2)

    # run at 10 MHz:
    target.pll.pll_outfreq_set(10E6, 1)

    # 1ms is plenty of idling time
    target.clkusbautooff = True
    target.clksleeptime = 1

In [5]:
if TARGET_PLATFORM in ['CW305_100t', 'CW305_35t']:
    if scope._is_husky:
        scope.clock.clkgen_freq = 10e6
        scope.clock.clkgen_src = 'extclk'
        scope.clock.adc_mul = 4
        # if the target PLL frequency is changed, the above must also be changed accordingly
    else:
        scope.clock.adc_src = "extclk_x4"
        

In [6]:
if TARGET_PLATFORM in ['CW312T_A35', 'CW312T_ICE40']:
    scope.clock.clkgen_freq = 7.37e6
    scope.io.hs2 = 'clkgen'
    if scope._is_husky:
        scope.clock.clkgen_src = 'system'
        scope.clock.adc_mul = 4
        scope.clock.reset_dcms()
    else:
        scope.clock.adc_src = "clkgen_x4"
    import time
    time.sleep(0.1)
    target._ss2_test_echo()

In [7]:
import time
for i in range(5):
    scope.clock.reset_adc()
    time.sleep(1)
    if scope.clock.adc_locked:
        break 
assert (scope.clock.adc_locked), "ADC failed to lock"

In [9]:
project_file = "projects/polymult_vulnerable.cwp"
project = cw.create_project(project_file, overwrite=True)

In [ ]:
from tqdm.notebook import tnrange
import numpy as np
import time
from Crypto.Cipher import AES

ktp = cw.ktp.Basic()

traces = []
textin = []
keys = []
N = 5000  # Number of traces

# initialize cipher to verify DUT result:
key, text = ktp.next()
cipher = AES.new(bytes(key), AES.MODE_ECB)

CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')
CWbytearray(b'77 a5 1c f4 6b ac 9f 37 60 2f 23 e4 7f 32 04 a8')
CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')
CWbytearray(b'36 c4 c0 c8 0f b2 54 a3 fc 6a b4 0c 13 50 a9 af')
CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')
CWbytearray(b'cf 37 4f 34 20 f7 e9 ef 2e 4c 17 f1 2a 3b 02 34')
